## Demand Forecasting and Markdown Pricing

## Problem statement
A retailer would like to decide weekly pricing of their products. He/she generally plans pricing for each quarter and would like to achieve following:
- Clear-out at least x% (say 60%) of total inventory
- Achieve total margin across all products (i.e., selling price - cost price)
Following are a few things to consider while deciding pricing of products.
- There shouldn’t be much difference in the pricing from one week to another.
- A particular price selected applies for all the products in the group.
- Shopkeeper must cater posed weekly demand to customers if inventory is available

## Tasks
- Develop a model that can help shopkeeper to identify their pricing for a quarter (13 weeks in the provided example)
- Model must suggest a pricing (price-point) that will be applied across all the products to achieve highest possible margin considering inventory clearance (say 60%)
- Max Price change from one week to next is 20 units
- A single price-point for a week that applies to all the products as they belong to same group
- Algorithmic details to solve the model and provide insights on complexity of model in scaling to larger number of products/weeks.

## Schema description
- product_id: 
    + unique product ID
- price: 
    + historical price in a week
- week: 
    + week in a quarter
- group: 
    + product group that a product belongs to (only one group in this case)
- demand: 
    + inventory sold during week at given price point
- selling_price: 
    + selling price of one unit of product in a week at a price-point
- total_inventory: 
    + Inventory available for each product at the start of the quarter. Same value is repeated for all possible weeks & price-points of a product. You can filter inventory available for a product by taking distinct values of columns product_id & total_inventory.
- cost_price: 
    + cost price of one unit of product in a week at a price-point

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from data import DATA
from preprocess import data_preprocess as dp
from forecast import SALES
from bayesian_opt import OPT, objective, bounds

In [ ]:
# Data = DATA()
# Data.get_data()
# Data.get_new_featuers()
# Data.merge_features()

# dp()

In [4]:
df_train = pd.read_csv("data/data_pre_processed.csv")
df_train

,Week,Cost_Price,Selling_Price,Total_Inventory,Demand
0,5,1015.44,2572.00,2107,947
1,6,924.24,1936.48,2107,950
2,7,958.32,2514.00,2107,944
3,8,1119.12,2561.00,2107,943
4,9,1116.00,2625.00,2107,953
...,...,...,...,...,...
10239,35,1931.76,3638.00,1796,508
10240,36,2015.04,2834.80,1796,520
10241,37,1799.28,3717.00,1796,520
10242,40,1645.20,3617.00,1708,507


In [5]:
df_train.describe()

,Week,Cost_Price,Selling_Price,Total_Inventory,Demand
count,10244.000000,10244.000000,10244.000000,10244.000000,10244.000000
mean,25.840785,1637.387114,3166.502120,1978.429324,861.085709
std,14.167845,340.876565,475.954817,99.443312,223.675528
min,1.000000,849.600000,1936.480000,1708.000000,500.000000
25%,14.000000,1386.960000,2719.000000,1910.000000,544.000000
50%,26.000000,1671.360000,3285.000000,2024.000000,962.000000
75%,38.000000,1931.520000,3594.000000,2037.000000,1037.000000
max,52.000000,2199.600000,3907.000000,2107.000000,1077.000000


In [6]:
Sales = SALES()
Sales.build()

Data Splitting Successful..
Model training started..
Model training successful..
Accuracy: 84.66 %


In [8]:
data = df_train.iloc[10:11,:-1]
data

,Week,Cost_Price,Selling_Price,Total_Inventory
10,15,1591.68,2808.0,2030


In [12]:
def objective(**args2):
    data["Selling_Price"] = args2["Selling_Price"]
    pred = Sales.model.predict(data.values)
    obj = - ( max(pred[0] - max((np.array(data["Total_Inventory"])*0.06) - pred[0],0), 0)) + 0.1*np.std(pred[0])
    return obj

bounds = {'Selling_Price':(1900,3900)}

In [19]:
result = dict()
for i in [10, 15, 22, 1300]:
    data = df_train.iloc[i:i+1,:-1]

    Opt = OPT(objective_fun=objective, bounds=bounds,verbose=2)
    Opt.run(n_iter=10)

    result[i] = Opt.optimizer.max
result

|   iter    |  target   | Sellin... |
-------------------------------------
|  1        | -965.8    |  2.734e+0 |
|  2        | -1.047e+0 |  3.341e+0 |
|  3        | -965.8    |  2.735e+0 |
|  4        | -940.4    |  1.977e+0 |
|  5        | -940.2    |  2.261e+0 |
|  6        | -550.6    |  3.9e+03  |
|  7        | -534.3    |  3.77e+03 |
|  8        | -538.5    |  3.828e+0 |
|  9        | -520.5    |  3.683e+0 |
|  10       | -511.2    |  3.61e+03 |
|  11       | -512.9    |  3.637e+0 |
|  12       | -720.3    |  3.577e+0 |
{'target': -511.1906590245281, 'params': {'Selling_Price': 3610.3592032409506}}
|   iter    |  target   | Sellin... |
-------------------------------------
|  1        | -966.5    |  2.734e+0 |
|  2        | -1.047e+0 |  3.341e+0 |
|  3        | -966.5    |  2.735e+0 |
|  4        | -940.4    |  1.977e+0 |
|  5        | -940.0    |  2.26e+03 |
|  6        | -550.4    |  3.9e+03  |
|  7        | -533.4    |  3.77e+03 |
|  8        | -537.6    |  3.828e+0 |
|  9    

{10: {'target': -511.1906590245281,
  'params': {'Selling_Price': 3610.3592032409506}},
 15: {'target': -510.9685417089279,
  'params': {'Selling_Price': 3611.0390188131896}},
 22: {'target': -508.6270647074353,
  'params': {'Selling_Price': 3608.7308526636243}},
 1300: {'target': -511.3005938977456,
  'params': {'Selling_Price': 3610.886625841008}}}